In [ ]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../data/nyc_subway_turnstile_sample.csv",index_col='time')
df.shape

In [ ]:
def preprocess_data(df, forecast_lead=15, train_test_split=0.8):
    features = list(df.columns)
    #target = f"{target_feature}_lead_{forecast_lead}"

    #df[target] = df[target_feature].shift(-forecast_lead)
    #df = df.iloc[:-forecast_lead]

    test_start = int(len(df) * train_test_split)

    df_train = df.iloc[:test_start].copy()
    df_test = df.iloc[test_start:].copy()

    #target_mean = df_train[target].mean()
    #target_stdev = df_train[target].std()

    for c in df_train.columns:
        mean = df_train[c].mean()
        stdev = df_train[c].std()

        df_train[c] = (df_train[c] - mean) / stdev
        df_test[c] = (df_test[c] - mean) / stdev

    return df_train, df_test, features

In [ ]:
class SequenceDataset(Dataset):
    def __init__(self, dataframe, features, sequence_length=5, forecast_lead=15):
        self.features = features
        self.forecast_lead = forecast_lead
        self.sequence_length = sequence_length
        self.X = torch.tensor(dataframe[features].iloc[:-15,:].values).float()
        self.y = torch.tensor(dataframe[features].iloc[self.forecast_lead:,:].values).float()
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, i):
        if i > self.sequence_length -1:
            i_start = i - self.sequence_length +1
            x = self.X[i_start:(i + 1), :]
        else:
            padding = self.X[0].repeat(self.sequence_length - i - 1, 1)
            x = self.X[0:(i+1), :]
            x = torch.cat((padding, x), 0)
        #y = self.X[i + self.forecast_lead]
        return x, self.y[i]

In [ ]:
SEQUENCE_LENGTH = 10
FORECAST_LEAD = 5
BATCH_SIZE = 32

df_train, df_test, features = preprocess_data(df,
                                              forecast_lead=FORECAST_LEAD,
                                              train_test_split=0.8
                                             )

In [ ]:
df_train.shape

In [ ]:
train_dataset = SequenceDataset(df_train, 
                                features,
                                sequence_length=SEQUENCE_LENGTH,
                                forecast_lead=FORECAST_LEAD)

test_dataset = SequenceDataset(df_test, 
                               features,
                               sequence_length=SEQUENCE_LENGTH,
                               forecast_lead=FORECAST_LEAD)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
class LSTMRegression(nn.Module):
    def __init__(self, num_features, hidden_units, num_layers=1,dropout=0):
        super().__init__()
        self.num_features = num_features
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers,
            dropout=self.dropout
        )
        self.linear = nn.Linear(in_features=self.hidden_units, out_features=self.num_features)
        
    def forward(self,x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        _,(hn,_) = self.lstm(x, (h0, c0))
        out = self.linear(hn[0])#.flatten()
        return out

In [ ]:
model = LSTMRegression(
        num_features=len(features), 
        hidden_units=2,
        num_layers=1,
        dropout=0
        )#.to(device=device)

In [ ]:
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)

In [ ]:
def score_model(data_loader, model, loss_function, device=torch.device("mps")):
    num_batches = len(data_loader)
    total_loss = 0
    #model = model.to(device)
    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            #X = X.to(device)
            #y = y.to(device)
            output = model(X)
            total_loss += loss_function(output, y).item()
    avg_loss = total_loss/num_batches
    #print(f"Test loss: {avg_loss}")
    return avg_loss

In [ ]:
test_score = score_model(test_loader, model, loss_function)

In [ ]:
test_score

In [ ]:
def train_model(data_loader, model, loss_function, optimizer, device=torch.device("mps")):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        #X = X.to(device=device)
        #y = y.to(device=device)
        output = model(X)
        loss = loss_function(output, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss/num_batches
    #print(f"Train loss: {avg_loss}")
    return avg_loss

In [ ]:
num_epochs = 200
for ix_epoch in range(num_epochs):
    #logger.info(f"Epoch: {ix_epoch}")
    train_score = train_model(train_loader, model, loss_function, optimizer=optimizer)
    #tb.add_scalar("Train Loss", train_score, ix_epoch)
    #logger.info(f"Train score: {train_score}")
    test_score = score_model(test_loader, model, loss_function)
    #tb.add_scalar("Test Loss", test_score, ix_epoch)
    #logger.info(f"Test score: {test_score}")
    print(f"Epoch {ix_epoch} -- Train Loss: {train_score}; Test Loss: {test_score}")

In [ ]:
def predict(data_loader, model):

    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    return output

def get_predictions(data_loader,model, df_test, target):
    ystar_col = "Model Forecast"
    #df_train[ystar_col] = predict(train_eval_loader, model).numpy()
    df_test[ystar_col] = predict(data_loader, model).numpy()

    df_out = df_test[[target, ystar_col]]

    #for c in df_out.columns:
    #    df_out[c] = df_out[c] * target_stdev + target_mean
    
    return df_out

In [ ]:
preds = predict(test_loader, model)

In [ ]:
preds.shape

In [ ]:
def plot_predictions(df_preds):
    fig_dims = (40, 10)
    fig, ax = plt.subplots(figsize=fig_dims)
    sns.lineplot(data=df_preds,ax=ax)
    plt.show()

In [1]:
def get_predictions(data_loader, model, df_test):
    preds = predict(data_loader, model)
    df_preds = pd.DataFrame()
    df_preds['y_pred'] = preds[:,0]
    df_preds['y_true'] = [x for x in df_test.iloc[:-15,0]]
    return df_preds

In [ ]:
plot_predictions(df_preds.head(410))